NLP has the goal of deriving information. out of natural language(could be sequences text or speech)

In [ ]:
#using the helper funtions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
# Import helper functions that can be used
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys,unzip_data

#Getting text dataset
The data is from kaggle's introduction to NLP dataset(labeled as disaster or not disaster)

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [ ]:
unzip_data("nlp_getting_started.zip")

#Visualizing a text dataset

to visualize our text samples, we first have to read them in, one way to do so would be to ue Python: https://realpython.com/read-write-files-python/

another way is using pandas.


In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

In [ ]:
train_df["text"][0]

In [ ]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac =1 , random_state =42)
train_df_shuffled.head()

In [ ]:
test_df.head()

In [ ]:
#how many example of each class?
train_df.target.value_counts()

In [ ]:
#total number of sample
len(train_df), len(test_df)

In [ ]:
#visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5) # create random index not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target =row
  print(f"Target:{target}","(realdisaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

#split data into training adn validation sets


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming your data is in the 'train_df_shuffled' DataFrame
# 'text' column contains the text and 'target' column contains the labels

# Split the dataset into training and validation sets (90% train, 10% validation)
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),  # Features (text)
    train_df_shuffled["target"].to_numpy(),  # Labels (target)
    test_size=0.1,  # 10% of the data will be used for validation
    random_state=42  # For reproducibility
)

# Check the shape of the resulting splits
print(f"Training sentences: {len(train_sentences)}")
print(f"Validation sentences: {len(val_sentences)}")
print(f"Training labels: {len(train_labels)}")
print(f"Validation labels: {len(val_labels)}")


In [ ]:
#use traintestsplit to split training data into training and validation sets
train_sentences,val_sentences,train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                          train_df_shuffled["target"].to_numpy(),
                                                                          test_size =0.1,#use 10 percent of training data for validation
                                                                          random_state =42)

In [ ]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

In [ ]:
#check the first 10 samples
train_sentences[:10], train_labels[:10]

# Converting text into numbers

ways to do it:
* Tokenization - direct mapping of token( could be word or character ) to number.
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and embeddings can be learned)


### Text vectorization(tokenization)

In [ ]:
train_sentences[:5]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [ ]:
#use the default Text vectorization parameters
max_vocab_length = 10000 # set maximum vocabulary size to 10000
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,# how many words in the vocabulary (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None, #Create groups of n-words?
                                    output_mode = "int",
                                    output_sequence_length = None, #None means it is going to find the longest sequence and pad to that length
                                    pad_to_max_tokens = True #pad to the maximum specified in output_sequence_length if its specified or to the longest one if left as default
                                    )

In [ ]:
train_sentences[0].split()

In [ ]:
train_sentences[0]

In [ ]:
#find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [ ]:
#setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 #max length of our sequences will be (e.g how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
#create a sample sentence and tokenize it
sample_sentence = "there's is a flood in my street!"
text_vectorizer([sample_sentence])

In [ ]:
#choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original text: \n {random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)

In [ ]:
bottom_5_words = words_in_vocab[-5:]


In [ ]:
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

# creating an embedding using embedding layer

the parameters we care most about for our embedding layer:
* 'input_dim' = the size of the vocabulary
* 'output_dim' = the size of the output embedding vector
* 'input_length' = length of the sequences being passed to the embedding layer

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length, #set input shape
                             output_dim = 128, #output shape
                             embeddings_initializer = "uniform",
                             input_length = max_length) #how long is each input
embedding

In [ ]:
# Get a random sentnce from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

In [ ]:
sample_embed[0][0]

In [ ]:
sample_embed[0][0].shape

In [ ]:
random_sentence

road map to machine learning https://scikit-learn.org/1.3/tutorial/machine_learning_map/

#Modelling a text dataset

* Model 0 : Naive Bayes(baseline)
* Model 1 : Feed-forward Neural Network(dense model)
* Model 2 : LSTM model(RNN)
* Model 3 : GRU model (RNN)
* Model 4 : Bidirectional-LSTM model(RNN)
* Model 5 : 1D convolutional Neural Network(CNN)
* Model 6 : Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7 : Same as model 6 with 10% of training data.



#Model 0 : Getting a baseline

To create our baseline, we'll use sklearn's multinomial naive bayes using the TF-IDF formula to convert words to numbers.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers using tfidf
    ("clf",MultinomialNB()) #model the text
])
#Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

In [ ]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

In [ ]:
train_df.target.value_counts()

In [ ]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

# Creating an evaluation function for our model experiments

* Accuracy
* Precision
* Recall
* F1-score


In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  """
  Calculates model accuracy,precision,recall and f1 score of a binary classification model
  """
  #calculate model accuracy
  model_accuracy = accuracy_score(y_true,y_pred)*100
  # calculate model precision, recall and F1-score
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {"accuracy":model_accuracy,
                   "precision":model_precision,
                   "recall":model_recall,
                   "f1":model_f1}
  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

# Model1 : A simple dense model


In [ ]:
# create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)# inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)# create an embedding of the numberized inputs
outputs = layers.Dense(1,activation = "sigmoid")(x) #create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")


In [ ]:
model_1.summary()

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)# inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)# create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vecotr for eact token to get single vector
outputs = layers.Dense(1,activation = "sigmoid")(x) #create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [ ]:
model_1.summary()

In [ ]:
#compile model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
#Fit the model
model_1_history = model_1.fit(x = train_sentences,
                                 y = train_labels,
                                 epochs = 5,
                                 validation_data = (val_sentences,val_labels),
                              callbacks= [create_tensorboard_callback(dir_name = SAVE_DIR,
                                          experiment_name = "model_1_dense")])

In [ ]:
# check the results
model_1.evaluate(val_sentences,val_labels)

In [ ]:
#make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape, model_1_pred_probs[0]

In [ ]:
#convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

In [ ]:
#calculate model1 results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results

In [ ]:
baseline_results

In [ ]:
import numpy as np
np.array(list(baseline_results.values())) > np.array(list(model_1_results.values()))

#visualizing the model embedding

In [ ]:
#Get the vocabulary from the text vectorization layer
words_in_vocab

In [ ]:
len(words_in_vocab)

In [ ]:
# Get the vocabulary from text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:10],len(words_in_vocab)

In [ ]:
#Model 1 summary
model_1.summary()

In [ ]:
#Get the weight matrix of embedding layer
#(these are the numerical representations of each token in our training data, which have been learned for f5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights


In [ ]:
print(embed_weights[0].shape)


# Visualize the embedding matrix which learned to represent our tokens using projector

study : https://jalammar.github.io/illustrated-word2vec/

work environment: https://projector.tensorflow.org/

In [ ]:
weights = model_1.get_layer("embedding").get_weights()[0]
vocab = text_vectorizer.get_vocabulary()

In [ ]:
#create embedding files (we got this from TensorFlow's word embeddings documentation)
import io
out_v = io.open("vectors.tsv","w",encoding = "utf-8")
out_m = io.open("metadata.tsv","w",encoding = "utf-8")
for index, word in enumerate(vocab):
  if index == 0:
    continue # skip 0 , it's padding
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec])+"\n")
  out_m.write(word+"\n")
out_v.close()
out_m.close()

In [ ]:
#create embedding files (we got this from TensorFlow's word embeddings documentation)
import io
out_v = io.open("vectors.tsv","w",encoding = "utf-8")
out_m = io.open("metadata.tsv","w",encoding = "utf-8")
for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0 , it's padding
  # the error was caused because the index was larger than the shape of the embed_weights
  # we need to ensure the index is less than the length of weights
  if index < embed_weights[0].shape[0]:
    vec = embed_weights[0][index]
    out_v.write('\t'.join([str(x) for x in vec])+"\n")
    out_m.write(word+"\n")
out_v.close()
out_m.close()

In [ ]:
#download files from colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

# Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.
it is to use the representation of previous input to aid the representation of
a later input.

### Model 2 : LSTM(Long Short Term Memory)



In [ ]:
#Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x)# when you're stacking RNN cells together, you need to return_sequence
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

In [ ]:
model_2.summary()

In [ ]:
#compile the model
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

In [ ]:
#Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

In [ ]:
#covert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
val_labels

In [ ]:
#Calculate model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

In [ ]:
baseline_results

## Model 3 : GRU

Another popular and effective RNN cell is the Gated Recurrent Unit (GRU) it has less parameters than LSTM.

In [ ]:
#Create an GRU model
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(64,return_sequences = True)(x)# when you're stacking RNN cells on top of each other, you need to return_sequence
print(x.shape)
x = layers.GRU(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [ ]:
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
#fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"model_3_GRU")])

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

In [ ]:
#convert model 3 predprobs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
# calculate model 3 results
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred = model_3_preds)
model_3_results

### Model 4: Bidirectional RNN

Normal RNN'S go from left to right
but this model go from left to right and right to left


In [ ]:
#Build the model
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name = "model_4_Bidirectional")

In [ ]:
model_4.summary()

In [ ]:
#compile the model
model_4.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
#fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"model_4_Bidirectional")])

In [ ]:
#predict the model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
#squeeze the the probs
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
#results
model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_preds)
model_4_results

# Convolutional Neural Networks for Text(and other types of sequences)
we've used CNNs for images are typically 2D but now it is in 1D

#Model 5 : Conv1D


In [ ]:
# Test ourt our embedding layer, conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) #turn target sequence into embedding
conv_1d = layers.Conv1D(filters = 32 ,
                        kernel_size = 5,
                        activation = "relu",
                        padding = "valid")
conv_1d_output = conv_1d(embedding_test)# pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)# "get the most important feature or get feature with highest value"
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape


In [ ]:
embedding_test

In [ ]:
conv_1d_output

In [ ]:
max_pool_output

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape= (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size = 5 , activation = "relu",padding = "valid")(x)
x  = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name = "model_5_Conv1D")

In [ ]:
#compile the model
model_5.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
#fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

In [ ]:
#Predict
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

In [ ]:
#results
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

In [ ]:
from helper_functions import calculate_results

In [ ]:
#results
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds)
model_5_results


#Model 6: TensorFlow Hub Pretrained Sentence Encoder


In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")
embed_samples = embed([
    sample_sentence,
    "I am a sentence for which I would like to get its embedding"])

print(embed_samples[0][:50])

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


In [ ]:
embed_samples

In [ ]:
# Load Universal Sentence Encoder (USE) from Kaggle Models
sentence_encoder_layer = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
    dtype=tf.string,
    trainable=False,
    name="USE_Layer"
)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model


# Define the Text Classification Model
def build_text_classifier(num_classes=2):
    """
    Creates a text classification model using USE as an embedding layer.

    Args:
        num_classes (int): Number of output classes (default: 2 for binary classification).

    Returns:
        model (tf.keras.Model): Compiled text classification model.
    """

    # Input layer (text data)
    text_input = layers.Input(shape=(), dtype=tf.string, name="text_input")

    # Wrap USE inside Lambda with explicit output shape
    embedding = layers.Lambda(lambda x: sentence_encoder_layer(x), output_shape=(512,), name="USE_Wrapper")(text_input)

    # Fully connected layers
    x = layers.Dense(128, activation="relu")(embedding)
    # Output layer
    if num_classes == 2:
        output = layers.Dense(1, activation="sigmoid", name="output_layer")(x)  # Binary classification
        loss = "binary_crossentropy"
    else:
        output = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)  # Multi-class classification
        loss = "sparse_categorical_crossentropy"

    # Create and compile model
    model = Model(inputs=text_input, outputs=output, name="text_classifier")
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model

# Create the text classification model
model_6 = build_text_classifier(num_classes=2)

# Model summary
model_6.summary()


In [ ]:
model_6.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

#transfer learning is not executing

#model 7: using 10% data


In [ ]:
train_10_percent = train_df_shuffled[["text","target"]].sample(frac = 0.1,random_state = 42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()

In [ ]:
#check the number of targets in our subset of data
train_10_percent["target"].value_counts()

In [ ]:
#results in dataframe
all_model_results = pd.DataFrame({"0_baseline":baseline_results,
                                 "1_simple_dense":model_1_results,
                                 "2_lstm": model_2_results,
                                 "3_GRU": model_3_results,
                                 "4_bidirectional": model_4_results,
                                 "5_conv1d": model_5_results})
all_model_results = all_model_results.transpose()


In [ ]:
all_model_results

In [ ]:
#reduce the accuracy to the same scale as other metrics
all_model_results["accuracy"]= all_model_results["accuracy"]*100

In [ ]:
#plot and compare all ot the model results
all_model_results.plot(kind = "bar",figsize=(10,7)).legend(bbox_to_anchor=(1.0,1.0));

In [ ]:
#sort model results by f1-score
all_model_results.sort_values("f1",ascending = False)["f1"].plot(kind="bar",figsize=(10,7));

In [ ]:
pip install google-cloud-storage


In [ ]:
import os
from getpass import getpass

# Enter your GitHub username
github_username = "varshith2415"

# Enter your GitHub token (you can paste it here, or use getpass for security)
github_token = getpass("Enter your GitHub token: ")

# Store authentication details securely
os.environ["GITHUB_USERNAME"] = github_username
os.environ["GITHUB_TOKEN"] = github_token



In [ ]:
!apt-get install git  # Install Git if not already installed

# Set up Git configuration (with your GitHub username and email)
!git config --global user.name "varshith2415"
!git config --global user.email "varshithreddyraavi@gmail.com"  # Replace with your email


In [ ]:
!git clone https://github.com/varshith2415/tensorflow-logs.git


In [ ]:
# Assuming the model_logs directory is in your current working directory in Colab
!cp -r /content/model_logs /content/tensorflow-logs/
